In [ ]:
# Ensure you have the required packages installed


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from imutils import paths
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# Load the CSV file with the training data
df_train = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv')
dirpath = '/kaggle/input/gtsrb-german-traffic-sign'

# Get the image paths from the dataframe
imagepaths = df_train['Path']
imagepaths = [os.path.join(dirpath, path) for path in imagepaths]

# Define a function to preprocess the data
def GetData(imagepaths):
    data = []
    labels = []
    for imagepath in tqdm(imagepaths,desc='processing images'):
        image = cv2.imread(imagepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (32, 32))  # Resize all images to 32x32
       
        label = imagepath.split('/')[-2]
        labels.append(label)
        data.append(image)
    
    data = np.array(data)
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    labels = to_categorical(labels)
    return data, labels

# Preprocess the data
data, labels = GetData(imagepaths)

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(data, labels, test_size=0.4, random_state=42)

# Print the shape of the first training sample to confirm it is processed correctly
print(xtrain[0].shape)


In [ ]:
plt.imshow(xtrain[0])

In [ ]:
from imutils import paths
import numpy as np
import tensorflow as tf
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class TrafficSignNet:
    
    def __init__(self, h, w, depth, classes):
        self.model = Sequential()
        chandim = -1
        self.model.add(Conv2D(8, (3, 3), padding='same', input_shape=(h, w, depth), activation='relu'))
        self.model.add(BatchNormalization(axis=chandim))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(16, (3, 3), padding='same', activation='relu'))
        self.model.add(BatchNormalization(axis=chandim))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
        self.model.add(BatchNormalization(axis=chandim))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
        self.model.add(BatchNormalization(axis=chandim))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.5))

        self.model.add(Dense(128, activation='relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.5))

        self.model.add(Dense(units=classes, activation='softmax'))

In [ ]:
aug = ImageDataGenerator(
rotation_range=10,
zoom_range=0.15,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.15,
horizontal_flip=False,
vertical_flip=False,
fill_mode="nearest")

In [ ]:
net = TrafficSignNet(h=32, w=32, depth=3, classes=43)
model = net.model 
model.compile(loss="categorical_crossentropy", optimizer='adam',
              metrics=["accuracy"])
history = model.fit(
    aug.flow(xtrain, ytrain, batch_size=32),
    validation_data=(xtest, ytest),
    epochs=15
)

In [ ]:
test_image = cv2.imread('/kaggle/input/gtsrb-german-traffic-sign/Test/00022.png')
test_image= cv2.resize(test_image,(32,32))
test_image = np.expand_dims(test_image,axis=0)
preds = model.predict(test_image)
print(preds)
print(np.argmax(preds))